# Day 7: Complete Interpretable ML Pipeline

## Learning Objectives
- Build end-to-end interpretable ML system
- Combine feature engineering, selection, and explainability
- Production-ready documentation and reporting

---

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import mutual_info_classif
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from sklearn.inspection import permutation_importance
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

print("✅ Libraries loaded!")
print("📚 Day 7: Complete Interpretable ML Pipeline")

## Part 1: Feature Engineering Module

In [ ]:
# ============================================================
# FEATURE ENGINEERING CLASS
# ============================================================

class FeatureEngineer:
    """Comprehensive feature engineering for financial data."""
    
    def __init__(self):
        self.feature_names = []
        
    def fit_transform(self, df):
        """Generate all features."""
        result = df.copy()
        
        # Return features
        for h in [1, 5, 10, 20]:
            result[f'ret_{h}d'] = df['price'].pct_change(h)
        
        # Volatility features
        for h in [5, 10, 20]:
            result[f'vol_{h}d'] = df['returns'].rolling(h).std()
        
        # Momentum features
        for h in [5, 10, 20]:
            result[f'mom_{h}d'] = df['returns'].rolling(h).sum()
        
        # RSI
        gains = df['returns'].where(df['returns'] > 0, 0).rolling(14).mean()
        losses = (-df['returns'].where(df['returns'] < 0, 0)).rolling(14).mean()
        result['rsi'] = 100 - (100 / (1 + gains / (losses + 1e-10)))
        result['rsi_norm'] = (result['rsi'] - 50) / 50
        
        # MA ratios
        result['ma_ratio_5_20'] = df['price'].rolling(5).mean() / df['price'].rolling(20).mean()
        
        # Volatility ratio
        result['vol_ratio'] = result['vol_5d'] / result['vol_20d']
        
        # Momentum acceleration
        result['mom_accel'] = result['mom_5d'] - result['mom_5d'].shift(5)
        
        self.feature_names = [c for c in result.columns if c not in ['price', 'returns']]
        
        return result
    
    def get_feature_names(self):
        return self.feature_names

print("✅ FeatureEngineer class defined")

## Part 2: Feature Selection Module

In [ ]:
# ============================================================
# FEATURE SELECTOR CLASS
# ============================================================

class FeatureSelector:
    """Feature selection with multiple methods."""
    
    def __init__(self, n_features=10, correlation_threshold=0.85):
        self.n_features = n_features
        self.corr_threshold = correlation_threshold
        self.selected_features = []
        self.selection_report = {}
        
    def fit(self, X, y, feature_names):
        """Select features using multiple methods."""
        
        # 1. Remove highly correlated features
        df_temp = pd.DataFrame(X, columns=feature_names)
        corr_matrix = df_temp.corr().abs()
        
        to_drop = set()
        for i in range(len(feature_names)):
            for j in range(i+1, len(feature_names)):
                if corr_matrix.iloc[i, j] > self.corr_threshold:
                    to_drop.add(feature_names[j])
        
        remaining = [f for f in feature_names if f not in to_drop]
        X_remaining = df_temp[remaining].values
        
        # 2. Mutual Information
        mi_scores = mutual_info_classif(X_remaining, y, random_state=42)
        mi_ranking = sorted(zip(remaining, mi_scores), key=lambda x: x[1], reverse=True)
        
        # 3. Select top features
        self.selected_features = [f for f, _ in mi_ranking[:self.n_features]]
        
        # Store report
        self.selection_report = {
            'original_features': len(feature_names),
            'after_corr_filter': len(remaining),
            'final_selected': len(self.selected_features),
            'dropped_correlated': list(to_drop),
            'mi_ranking': mi_ranking
        }
        
        return self
    
    def transform(self, X, feature_names):
        """Select only chosen features."""
        df_temp = pd.DataFrame(X, columns=feature_names)
        return df_temp[self.selected_features].values
    
    def get_selected_features(self):
        return self.selected_features
    
    def print_report(self):
        print("\nFEATURE SELECTION REPORT")
        print("="*50)
        print(f"Original features: {self.selection_report['original_features']}")
        print(f"After correlation filter: {self.selection_report['after_corr_filter']}")
        print(f"Final selected: {self.selection_report['final_selected']}")
        print(f"\nSelected features: {self.selected_features}")

print("✅ FeatureSelector class defined")

## Part 3: Model Explainer Module

In [ ]:
# ============================================================
# MODEL EXPLAINER CLASS
# ============================================================

class ModelExplainer:
    """Model explainability toolkit."""
    
    def __init__(self, model, feature_names):
        self.model = model
        self.feature_names = feature_names
        self.shap_values = None
        self.permutation_importance = None
        
    def calculate_shap(self, X_train, X_test, n_samples=100):
        """Calculate SHAP-like values."""
        n_explain = min(n_samples, len(X_test))
        n_features = X_test.shape[1]
        
        shap_values = np.zeros((n_explain, n_features))
        
        for i in range(n_explain):
            x = X_test[i:i+1]
            original_pred = self.model.predict_proba(x)[0, 1]
            
            for j in range(n_features):
                x_modified = x.copy()
                x_modified[0, j] = X_train[:, j].mean()
                modified_pred = self.model.predict_proba(x_modified)[0, 1]
                shap_values[i, j] = original_pred - modified_pred
        
        self.shap_values = shap_values
        return shap_values
    
    def calculate_permutation_importance(self, X_test, y_test):
        """Calculate permutation importance."""
        result = permutation_importance(self.model, X_test, y_test, n_repeats=20, random_state=42)
        self.permutation_importance = pd.DataFrame({
            'feature': self.feature_names,
            'importance': result.importances_mean,
            'std': result.importances_std
        }).sort_values('importance', ascending=False)
        return self.permutation_importance
    
    def explain_prediction(self, idx, X_test, y_test):
        """Explain a single prediction."""
        sample = X_test[idx:idx+1]
        pred_prob = self.model.predict_proba(sample)[0, 1]
        actual = y_test[idx]
        
        print(f"\nPREDICTION EXPLANATION (Sample {idx})")
        print("="*50)
        print(f"Prediction: {pred_prob:.2%} probability of UP")
        print(f"Actual: {'UP ✓' if actual == 1 else 'DOWN ✗'}")
        
        if self.shap_values is not None and idx < len(self.shap_values):
            contrib = pd.DataFrame({
                'feature': self.feature_names,
                'value': sample[0],
                'contribution': self.shap_values[idx]
            }).sort_values('contribution', key=abs, ascending=False)
            
            print("\nTop contributors:")
            for _, row in contrib.head(5).iterrows():
                sign = '+' if row['contribution'] > 0 else '-'
                print(f"  {row['feature']:<15} = {row['value']:>7.3f}  {sign}{abs(row['contribution']):.4f}")
    
    def plot_summary(self, X_test):
        """Create summary visualizations."""
        fig, axes = plt.subplots(1, 2, figsize=(14, 5))
        
        # Permutation importance
        if self.permutation_importance is not None:
            self.permutation_importance.plot(x='feature', y='importance', kind='barh',
                                             xerr=self.permutation_importance['std'],
                                             ax=axes[0], color='steelblue', alpha=0.7, legend=False)
            axes[0].set_xlabel('Mean Accuracy Decrease')
            axes[0].set_title('Permutation Importance', fontweight='bold')
            axes[0].invert_yaxis()
        
        # SHAP summary
        if self.shap_values is not None:
            mean_shap = np.abs(self.shap_values).mean(axis=0)
            sorted_idx = np.argsort(mean_shap)[::-1]
            
            for i, feat_idx in enumerate(sorted_idx[:8]):
                shap_vals = self.shap_values[:, feat_idx]
                feat_vals = X_test[:len(shap_vals), feat_idx]
                norm_feat = (feat_vals - feat_vals.min()) / (feat_vals.max() - feat_vals.min() + 1e-10)
                
                y_jitter = i + np.random.randn(len(shap_vals)) * 0.15
                axes[1].scatter(shap_vals, y_jitter, c=norm_feat, cmap='coolwarm', alpha=0.5, s=10)
            
            axes[1].set_yticks(range(min(8, len(self.feature_names))))
            axes[1].set_yticklabels([self.feature_names[i] for i in sorted_idx[:8]])
            axes[1].axvline(0, color='gray', linewidth=0.5)
            axes[1].set_xlabel('SHAP Value')
            axes[1].set_title('SHAP Summary', fontweight='bold')
            axes[1].invert_yaxis()
        
        plt.tight_layout()
        plt.show()

print("✅ ModelExplainer class defined")

## Part 4: Complete Pipeline

In [ ]:
# ============================================================
# GENERATE DATA
# ============================================================

print("RUNNING COMPLETE PIPELINE")
print("="*60)

# Generate data
np.random.seed(42)
n_days = 2000

returns = np.random.normal(0.0002, 0.015, n_days)
for i in range(1, len(returns)):
    returns[i] += 0.05 * returns[i-1]

prices = 100 * np.cumprod(1 + returns)
df = pd.DataFrame({'price': prices, 'returns': returns})

print(f"Generated {n_days} days of data")

In [ ]:
# ============================================================
# STEP 1: FEATURE ENGINEERING
# ============================================================

print("\nSTEP 1: FEATURE ENGINEERING")
print("-"*50)

fe = FeatureEngineer()
df_features = fe.fit_transform(df)

# Add target
df_features['target'] = (df['returns'].shift(-1) > 0).astype(int)
df_features = df_features.dropna()

feature_names = fe.get_feature_names()
X = df_features[feature_names].values
y = df_features['target'].values

print(f"Created {len(feature_names)} features: {feature_names}")

In [ ]:
# ============================================================
# STEP 2: FEATURE SELECTION
# ============================================================

print("\nSTEP 2: FEATURE SELECTION")
print("-"*50)

selector = FeatureSelector(n_features=8, correlation_threshold=0.85)
selector.fit(X, y, feature_names)

X_selected = selector.transform(X, feature_names)
selected_features = selector.get_selected_features()

selector.print_report()

In [ ]:
# ============================================================
# STEP 3: MODEL TRAINING
# ============================================================

print("\nSTEP 3: MODEL TRAINING")
print("-"*50)

# Scale and split
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_selected)

split = int(len(X_scaled) * 0.7)
X_train, X_test = X_scaled[:split], X_scaled[split:]
y_train, y_test = y[:split], y[split:]

# Train model
model = GradientBoostingClassifier(n_estimators=100, max_depth=4, random_state=42)
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
y_proba = model.predict_proba(X_test)[:, 1]

print(f"\nModel Performance:")
print(f"  Accuracy:  {accuracy_score(y_test, y_pred):.3f}")
print(f"  Precision: {precision_score(y_test, y_pred):.3f}")
print(f"  Recall:    {recall_score(y_test, y_pred):.3f}")
print(f"  F1:        {f1_score(y_test, y_pred):.3f}")
print(f"  AUC-ROC:   {roc_auc_score(y_test, y_proba):.3f}")

In [ ]:
# ============================================================
# STEP 4: MODEL EXPLAINABILITY
# ============================================================

print("\nSTEP 4: MODEL EXPLAINABILITY")
print("-"*50)

explainer = ModelExplainer(model, selected_features)

# Calculate importance metrics
perm_imp = explainer.calculate_permutation_importance(X_test, y_test)
shap_vals = explainer.calculate_shap(X_train, X_test, n_samples=200)

print("\nPermutation Importance:")
print(perm_imp.to_string(index=False))

In [ ]:
# ============================================================
# STEP 5: VISUALIZE & EXPLAIN
# ============================================================

print("\nSTEP 5: VISUALIZATIONS")
print("-"*50)

explainer.plot_summary(X_test)

In [ ]:
# ============================================================
# INDIVIDUAL PREDICTIONS
# ============================================================

print("\nEXPLAINING INDIVIDUAL PREDICTIONS")
print("="*60)

explainer.explain_prediction(50, X_test, y_test)
explainer.explain_prediction(100, X_test, y_test)

In [ ]:
# ============================================================
# TRADING SIMULATION
# ============================================================

print("\nTRADING SIMULATION")
print("="*60)

# Get actual returns for test period
test_returns = returns[split + 60:split + 60 + len(y_test)]  # Offset for feature lag

min_len = min(len(test_returns), len(y_pred))
test_returns = test_returns[:min_len]
predictions = y_pred[:min_len]

# Strategy returns
strategy_returns = test_returns * (2 * predictions - 1)

# Metrics
cum_strategy = np.cumprod(1 + strategy_returns)
cum_market = np.cumprod(1 + test_returns)

strategy_sharpe = np.sqrt(252) * strategy_returns.mean() / strategy_returns.std()
market_sharpe = np.sqrt(252) * test_returns.mean() / test_returns.std()

print(f"Strategy Total Return: {(cum_strategy[-1]-1)*100:.1f}%")
print(f"Market Total Return:   {(cum_market[-1]-1)*100:.1f}%")
print(f"Strategy Sharpe:       {strategy_sharpe:.2f}")
print(f"Market Sharpe:         {market_sharpe:.2f}")

# Plot
fig, ax = plt.subplots(figsize=(12, 5))
ax.plot(cum_strategy, label=f'Strategy ({(cum_strategy[-1]-1)*100:.0f}%)', linewidth=2)
ax.plot(cum_market, label=f'Market ({(cum_market[-1]-1)*100:.0f}%)', alpha=0.7)
ax.set_xlabel('Trading Days')
ax.set_ylabel('Cumulative Return')
ax.set_title('Strategy vs Market Performance', fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
print("""
╔══════════════════════════════════════════════════════════════════╗
║      DAY 7 COMPLETE: INTERPRETABLE ML PIPELINE                   ║
╠══════════════════════════════════════════════════════════════════╣
║  ✓ FeatureEngineer: Comprehensive feature creation              ║
║  ✓ FeatureSelector: Correlation + MI-based selection            ║
║  ✓ Model Training: Gradient Boosting with evaluation            ║
║  ✓ ModelExplainer: SHAP + Permutation importance                ║
║  ✓ Individual prediction explanations                           ║
║  ✓ Trading simulation with performance metrics                  ║
╚══════════════════════════════════════════════════════════════════╝

🎉 WEEK 11 COMPLETE! You've mastered Feature Engineering & Explainability!

Next: Week 12 - Advanced Backtesting & Deployment
""")